# Giant Mess - Dealing with Timezones and Daylight Saving Time in Pandas
## - I went to sleep at 2 am on November 1... - Um, which one?
<img src='images/time.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@andrey-grushnikov-223358?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Andrey Grushnikov</a>
        on 
        <a href='https://www.pexels.com/photo/black-and-white-photo-of-clocks-707676/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels</a>
    </strong>
</figcaption>

### Time Zones

The first mechanical clock was invented in the 17th century. During those times, everyone set their clock according to noon and this was not a big problem because everyone traveled on foot and horses. Later, telegraphs and trains were invented and you could suddenly travel around and communicate with people hundreds of miles away in a significantly short amount of time. This led to people to realize they had to have a standard time so, in 1900s, the greatest leaders and thinkers from developed countries gathered around and divided the world into 24 time zones. 

Greenwich, London got be 0 o'clock because they standardized their time first and the countries west to the UK had their clocks behind and the East countries were ahead of Greenwich time. This standardization is knows as UTC (Universal Coordinate Time). 
In other words, if you Google 'UTC time now', it will show the time in London.

Today, data moves around the world in the blink of an eye and if date and time information is not timezone aware, it will create a whole host of problems for programmers. In this post, we will talk about how to address such problems, including Daylight Saving Time.